In [6]:
import cProfile 
import pstats
from enhanced_lib.exchange import ExchangeCache,Database
from enhanced_lib.exchange.types import TradeEntry

db = Database('http://localhost:35969')
# db = Database('http://localhost:8000')

# await db.generate_and_save_future_trades('main_account','BTCUSDT','gbozee1_sub_account')
account = 'gbozee1_sub_account'
exchange = await db.get_initialized_exchange(account,'BTCUSDT',account,True)
# exchange.open_orders.get_sl_orders('short',True)
# uu = await exchange.account.get_trades(exchange.symbol)
# kind = 'short'
# index = -1
# uu[kind][index]
# longs = [TradeEntry(**x) for x in uu[kind]]
# longs
# longs[index].trades
# for i in longs[index].trades:
#     if i.get('take_profit'):
#         print(i)
# longs[index].trade_instances
# target = 42460.4
# longs[index].new_entry_at(41054.12)
# longs[index].last_placed_trade(.077)
# # longs[index].pnl_at_sell(42460.4)

# longs[index].avg_entry(41054.12)
exchange.open_orders.get_open_orders('short',True,449.362)
# with cProfile.Profile() as profile:
#     db.generate_future_trades(
#         exchange,
#         kind=None,
#         full=True,
#         no_of_cpu=1,
#         ignore=True,
#     )
#     # profile.print_stats()
# results = pstats.Stats(profile)
# results.sort_stats(pstats.SortKey.TIME)
# results.print_stats()
# results.dump_stats('profile.prof')
# exchange.open_orders.get_tp_orders('short',True)

{'size': 0.358,
 'count': 8,
 'avg': 43007.661852,
 'last': 43405.1,
 'fees': 1.569941,
 'liquidation': 44086.516901,
 'loss': -142.282857}

In [1]:
from enhanced_lib.calculations.shared import build_config
from enhanced_lib.calculations.workers.optimum_risk_reward import eval_func,determine_optimum_reward,run_in_parallel
from enhanced_lib.calculations.workers.utils import chunks_in_threads,run_in_threads
app_config = future_instance.config.app_config
app_config.entry = 41993.9
app_config.stop = 44779.3
app_config.kind = 'short'
app_config.risk_reward = 80
# app_config.risk_per_trade = 70
print('app_config',app_config)
# app_config.support = 41900
# app_config.resistance  = 45000
# signal = build_config(app_config,{
#     # 'raw_instance':True,
#     'decimal_places':app_config.decimal_places,
#     'increase':True,
#     'risk_reward':80,
#     'risk_per_trade': 6,
#     'entry': 44200,
#     'kind': 'long',
#     'stop': 42000,
#     'currentEntry':44200
# })
# signal
# # signal.get_bulk_trade_zones(44200,kind='long')
app_config.strategy = 'entry'
# eval_func(80,app_config)
# for i in range(30,199,1):
#     uu = eval_func(i,app_config)
#     # print('uu',len(uu['result']))
# func = run_in_parallel(eval_func, [(x,app_config) for x in range(30,199,1)],no_of_cpu=8)
# func
# bb = chunks_in_threads(eval_func, [(x,app_config) for x in range(30,199,1)],no_of_cpu=4,num_threads=40)
# len(bb)
# func = run_in_threads(eval_func, [(x,app_config) for x in range(30,199,1)],num_threads=2)
# determine_optimum_reward(app_config)
# exchange.future_instance.config.long_liquidation_price
# exchange.get_next_tradable_zone('long')
# exchange.get_next_tradable_zone('short')
exchange.config_params_for_future_trades('short',True)

NameError: name 'future_instance' is not defined

In [14]:
from enhanced_lib.calculations.trade_signal import Signal,_get_zones,_get_zone_nogen
import cProfile



# [x for x in _get_zones(43500,18000,0.04,places='%.1f')]
with cProfile.Profile() as profile:
    _get_zone_nogen(43500,18000,0.04,places='%.1f')
    # [x for x in _get_zone_nogen(43500,18000,0.04,places='%.1f')]
    profile.print_stats()

         43 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 cProfile.py:40(print_stats)
        1    0.000    0.000    0.000    0.000 cProfile.py:50(create_stats)
        1    0.000    0.000    0.000    0.000 pstats.py:107(__init__)
        1    0.000    0.000    0.000    0.000 pstats.py:117(init)
        1    0.000    0.000    0.000    0.000 pstats.py:136(load_stats)
        1    0.000    0.000    0.000    0.000 trade_signal.py:24(_get_zone_nogen)
       11    0.000    0.000    0.000    0.000 utils.py:54(to_f)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
       12    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
       11    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.00